In [17]:
import pandas as pd
from itertools import product

In [18]:
experience_df = pd.read_csv("./datasets/formats/ias-experience.csv")
education_df = pd.read_csv("./datasets/formats/ias-education.csv")

In [19]:
subjects_df = pd.read_csv("./processed/IAS subjects.csv")

In [20]:
education_df = education_df.loc[education_df["Subject"] != "N.A."]
experience_df = experience_df.loc[experience_df["Category_of_Experience"] != "N.A."]

In [21]:
education_df.dropna(inplace=True)
experience_df.dropna(inplace=True)

In [22]:
# copy the subject columns to a list
subjects_items = education_df["Subject"].copy()
for subject, category in zip(subjects_df["Subject"], subjects_df["Category"]):
    subjects_items.replace({subject: category}, inplace=True)

In [23]:
education_df["Category"] = subjects_items

In [24]:
names = education_df["Name"].unique()

In [25]:
from tqdm import tqdm

In [26]:
subject_to_exp_map = []
for name in names:
    # filter the education and experience dataframe for the current name
    temp_edu = education_df.loc[education_df["Name"] == name]
    temp_exp = experience_df.loc[experience_df["Name"] == name]

    # get the unique subject categories
    subject_list = temp_edu["Category"].unique().tolist()

    # get the unique experience categories
    exp_list = temp_exp["Category_of_Experience"].apply(str).map(str.strip).unique().tolist()

    # get the unique combinations of subject and experience
    products = list(product(subject_list, exp_list))

    # add the name and the list of combinations to the map
    subject_to_exp_map.extend(products)

100%|██████████| 7105/7105 [00:43<00:00, 161.99it/s]


In [27]:
temp = pd.Series(subject_to_exp_map)
count_df = pd.DataFrame(temp.value_counts())

In [29]:
subject = []
experience = []

for s, e in count_df.index:
    subject.append(s)
    experience.append(e)

In [30]:
count_df["Subject"] = subject
count_df["Experience"] = experience
count_df.to_csv("./processed/unigram maps.csv", index=False)